In [17]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

#### Dataset functions

In [18]:
import os
import pandas as pd
import SimpleITK as sitk

# this data loader is in a file now
class Abus23DataLoader():
    def __init__(self, dataset_path, labels_csv=None):
        self.data = self.load_abus23(dataset_path, labels_csv)
        self.dataset_path = dataset_path
        self.used_data = self.data
        self.cidx = 0
        
    def load_abus23(self, dataset_path, label_file=None):
        if label_file is not None:
            dataset = pd.read_csv(os.path.join(dataset_path, label_file))
        else:
            files = sorted([f for f in os.listdir(dataset_path) if f.endswith(".nrrd")])
            files_id = [int(f.split(".")[0].split("_")[-1]) for f in files]
            files_path = [f for f in files] #os.path.join(dataset_path, f) 
            dataset_dict = [{"case_id": f_id, "data_path": f_path} for f_id, f_path in zip(files_id, files_path)]
            dataset = pd.DataFrame.from_dict(dataset_dict)
        print("Dataset columns:", dataset.columns)
        return dataset
    
    def set_subset_ids(self, list_id = [], id_label = 'case_id'):
        if list_id:
            self.used_data = self.data[self.data[id_label].isin(list_id)]
        
    def get_data_entry(self, idx):    
        return self.used_data.iloc[idx]
    
    def get_item(self, idx):
        entry = self.get_data_entry(idx).to_dict()
        output = {}
        output["id"] = entry['case_id']
        if 'label' in entry:
            output["class"] =entry['label']
        image_full_path = os.path.join(self.dataset_path, entry['data_path'].replace('\\','/'))
        output["image"] = sitk.ReadImage(image_full_path)
        output["image_path"] = image_full_path
        if 'mask_path' in entry:
            mask_full_path = os.path.join(self.dataset_path, entry['mask_path'].replace('\\','/'))
            output["mask"] = sitk.ReadImage(mask_full_path)
            output["mask_path"] = mask_full_path
            
        return output

    def get_keys(self):
        return self.used_data.columns.tolist()
        
    def __getitem__(self, idx):
       return self.get_item(idx)
   
    def __len__(self):
        return len(self.used_data)
   
def get_validation_ids(val_file):
    with open(val_file) as fp:
        lines = fp.readlines()
        #print([os.path.basename(i)for i in lines])
        patients = set([int(os.path.basename(case).split('_')[0]) for case in lines])
    return list(patients)


#### Data processing functions

In [19]:

import SimpleITK as sitk
import numpy as np
import cv2

def normalize_8bits(image: np.ndarray):
    return (255.0 *(image - image.min()) / (image.max() - image.min())).astype(np.uint8)




# Create the volume from slices
def volume_from_slice(slices):
    mask_3d = np.stack(slices)
    return mask_3d

def sitk_from_volum(volum, format=None):
    output_mask = sitk.GetImageFromArray(volum)
    if format is not None:
        #sitk.GetPixelIDValueAsString(format)
        castImageFilter = sitk.CastImageFilter()
        castImageFilter.SetOutputPixelType(format)
        output_mask = castImageFilter.Execute(output_mask)
        
    return output_mask

def sitk_volume_from_slice(slices, format=None):
    mask_3d =  volume_from_slice(slices)
    output_mask = sitk_from_volum(mask_3d, format)
    caaa = sitk.GetArrayViewFromImage(output_mask)
    print("After sitk volum:",np.min(caaa ), np.max(caaa))
    
    return output_mask

def get_bounding_box(mask_image):
    # Convert SimpleITK image to NumPy array
    mask_array = sitk.GetArrayFromImage(mask_image)
    
    if sum(mask_array.flatten()) == 0:
        return np.zeros(3), np.zeros(3)
    
    # Get nonzero indices from the mask array
    nonzero_indices = np.nonzero(mask_array)
    
    # Calculate bounding box coordinates and dimensions
    min_coords = np.min(nonzero_indices, axis=1)
    max_coords = np.max(nonzero_indices, axis=1)
    center = (min_coords + max_coords)/2
    dimensions = max_coords - min_coords + 1
    
    return center, dimensions


def calculate_bb_score(image, bb):
    # Convert the cropped region to a NumPy array
    array = sitk.GetArrayViewFromImage(image)
    cropped_array = array[bb[2]:bb[2]+bb[5], bb[1]:bb[1]+bb[4], bb[0]:bb[0]+bb[3]]
    #cropped_array = array[bb[0]:bb[0]+bb[3], bb[1]:bb[1]+bb[4], bb[2]:bb[2]+bb[5]]
    print("After crop volum:",np.min(cropped_array), np.max(cropped_array))

    # Calculate the BB score by averaging non-zero values
    non_zero_values = cropped_array[cropped_array != 0]
    if non_zero_values.size > 0:
        bb_score = np.mean(non_zero_values)
    else:
        bb_score = 0.0

    return bb_score

def best_bb_bounding_boxes(mask_bn, mask_prob, th=0):
    
    caaa = sitk.GetArrayViewFromImage(mask_prob)
    print("Prob",np.min(caaa ), np.max(caaa))
    
    mradius=5
    #mask_bn = sitk.BinaryMorphologicalClosing(mask_bn, kernelRadius=[mradius]*3 )
    mask_bn = sitk.BinaryDilate(mask_bn, kernelRadius=[mradius]*3 )
    mask_cc = sitk.ConnectedComponent(mask_bn)
    
    label_stats = sitk.LabelShapeStatisticsImageFilter()
    label_stats.Execute(mask_cc)
    num_labels = label_stats.GetNumberOfLabels()
    print(num_labels)

    best_score = 0
    best_bb = [0,0,0], [0,0,0]
    for label in range(1, num_labels + 1):
        bbox = label_stats.GetBoundingBox(label)
        print("BB", bbox)
        score = calculate_bb_score(mask_prob, bbox)
        min_x, min_y, min_z, size_x, size_y, size_z = bbox
        print([min_x+size_x/2, min_y+size_y/2, min_z+size_z/2], [size_x, size_y, size_z], score)
        if score > th and score > best_score:
            best_score = score
            best_bb = [min_z+size_z/2, min_y+size_y/2, min_x+size_x/2], [size_z, size_y, size_x]
            #best_bb = [min_x+mradius, min_y+mradius, min_z+mradius], [size_x-mradius, size_y-mradius, size_z-mradius]
            
    return best_bb[0], best_bb[1], best_score

def binary_mask(image, output_format):
    greater_mask = sitk.Greater(image, 0)
    mask = sitk.Cast(greater_mask, output_format)
    caaa = sitk.GetArrayViewFromImage(mask)
    print("binary",np.min(caaa ), np.max(caaa))
    return mask



# Cropping
def get_slices(data, norm_fn = normalize_8bits):
    if isinstance(data, sitk.Image):
        data_array = sitk.GetArrayFromImage(data)
    if isinstance(data, np.ndarray):
        data_array = data
    return [norm_fn(data_array[i, ...]) for i in range(len(data_array))]

def crop_volume(data, crop=[0,0,0,0,0,0]):
    data_array = sitk.GetArrayFromImage(data)
    x1 = int(data_array.shape[2]*crop[0])
    y1 = int(data_array.shape[1]*crop[1])
    z1 = int(data_array.shape[0]*crop[2])
    x2 = int(data_array.shape[2]*crop[3] if crop[3] > 0 else -data_array.shape[2])
    y2 = int(data_array.shape[1]*crop[4] if crop[4] > 0 else -data_array.shape[1])
    z2 = int(data_array.shape[0]*crop[5] if crop[5] > 0 else -data_array.shape[0])
    print(data_array.shape)
    print(z1,z2, y1,y2, x1,x2)
    crop_array = data_array[z1:-z2, y1:-y2, x1:-x2]
    crop_pad = [(z1,z2 if crop[5] > 0 else 0),
                (y1,y2 if crop[4] > 0 else 0),
                (x1,x2 if crop[3] > 0 else 0)]
    return crop_array, crop_pad

def volume_from_slice_wpad(slices, format=None, pad=None):
    mask_3d = np.stack(slices)
    
    if pad is not None:
        mask_3d = np.pad(mask_3d, pad)
    
    output_mask = sitk.GetImageFromArray(mask_3d)
    
    if format is not None:
        #sitk.GetPixelIDValueAsString(format)
        castImageFilter = sitk.CastImageFilter()
        castImageFilter.SetOutputPixelType(format)
        output_mask = castImageFilter.Execute(output_mask)
        
    return output_mask

#### Prediction functions

In [20]:
from ultralytics import YOLO
import numpy as np
import json
import cv2

# Load a model
class YOLOPredictor:
    def __init__(self, model_file, conf_th = 0.5):
        self.model = YOLO(model_file)  # pretrained YOLOv8n model
        self.conf_th = conf_th
        
    def set_conf_th(self, conf_th = 0.5):
        self.conf_th = conf_th
        
    def __call__(self, slice, conf_th=None):
        return self.predict(slice, conf_th)
        
    def predict(self, slice, conf_th=None):
        assert len(slice.shape) == 2
        
        if conf_th is None:
            conf_th = self.conf_th
        
        #cv2.imwrite("temp.png", slice)
        #results = self.model("temp.png", verbose=False)[0].cpu().numpy()
        bgr_slice = cv2.cvtColor(slice, cv2.COLOR_GRAY2BGR)
        results = self.model(bgr_slice, verbose=False)[0].cpu().numpy()
        
        """ best only
        slice_mask = np.zeros(slice.shape)
        avg_conf = None
        if results.masks is not None:
            total_conf = []
            pred_mask_data = results.masks.data
            pred_box_conf = [results.boxes[i].conf for i in range(len(pred_mask_data))]
            best_pred = np.argmax(pred_box_conf)
            best_pred_conf = pred_box_conf[best_pred]

            if best_pred_conf >= conf_th:
                slice_mask = cv2.resize(pred_mask_data[best_pred, ...], dsize=(slice.shape[1], slice.shape[0])) # interpolation=cv2.INTER_CUBIC)
                avg_conf = pred_box_conf[best_pred]
        """
        
        """ All
        slice_mask = np.zeros(slice.shape)
        avg_conf = None
        if results.masks is not None:
            total_conf = []
            pred_mask_data = results.masks.data
            for i in range(len(pred_mask_data)):
                
                pred_box_conf = results.boxes[i].conf  # confidence score, (N, )
                total_conf.append(pred_box_conf)
                if pred_box_conf < conf_th:
                    continue

                m = cv2.resize(pred_mask_data[i, ...], dsize=(slice.shape[1], slice.shape[0])) # interpolation=cv2.INTER_CUBIC)
                slice_mask = np.logical_or(slice_mask, m).astype("float32")
            avg_conf = np.mean(total_conf) 
        """       
        
        # All w/prob_mask
        slice_mask = np.zeros(slice.shape, dtype=np.float32)
        if results.masks is not None:
            pred_mask_data = results.masks.data
            for i in range(len(pred_mask_data)):
                pred_box_conf = results.boxes[i].conf  # confidence score, (N, )
                if pred_box_conf < conf_th:
                    continue
                m = cv2.resize(pred_mask_data[i, ...], dsize=(slice.shape[1], slice.shape[0])) # interpolation=cv2.INTER_CUBIC)
                slice_mask[m>0] = pred_box_conf           
        
        return slice_mask  #, avg_conf

#### Run inference

In [21]:
from ultralytics import YOLO
from tqdm import tqdm

# ABUS 23 Test data
dataset_path = "/home/joel/abus23/datasets/DATA" 
label_file = None #"labels.csv"
validation_file = None #"datasets/abus23_25_png/val_seg.txt"

# ABUS 23 Train data
dataset_path = "/home/joel/abus23/datasets/Train" 
label_file = "labels.csv"
bbx_labels = "bbx_labels.csv"
validation_file = "/home/joel/abus23/datasets/abus23_25_png/val_seg.txt"

# Yolo model
yolo_weights = "/home/joel/abus23/runs/segment/train10/weights/best.pt"

# Volume
confidance_th = 0.6
crop = [0,0,0,0,0,0]

#crop = [0.03, 0.03, 0, 0.1, 0.03, 0.03]


# Output folder
output_folder = os.path.join( "results_masks", "abus23_25", "raw_stack_train10_param1")
output_file_template = "MASK_{:0>3}.nii.gz"
output_format = sitk.sitkUInt8

# CSV files
pred_csv = os.path.join(output_folder, "prediction.csv")
if "bbx_labels" in locals() and bbx_labels is not None:
        gt_csv = os.path.join(dataset_path, bbx_labels)


# Create output folder
os.makedirs(output_folder, exist_ok=False)

detection_results = []

# Load dataset
dataset = Abus23DataLoader(dataset_path, label_file)

# Get validation cases
if validation_file is not None:
        val_ids = get_validation_ids(validation_file)
        dataset.set_subset_ids(val_ids)

# Load yolo predictor
yolo_predictor = YOLOPredictor(yolo_weights)
# For each item in the dataset
gt_files = []
pred_files = []
for item in tqdm(dataset):
    
        cropped_image, crop_pad = crop_volume(item['image'], crop)
    
        # Get image slice
        image_slices = get_slices(cropped_image)

        # Get predicted slices
        yolo_masks_slices = [yolo_predictor(slice, conf_th=confidance_th) 
                             for slice in image_slices]

        # Create 3D volum
        prob_volum = volume_from_slice_wpad(yolo_masks_slices, sitk.sitkFloat32, pad=crop_pad)  
        bn_volum = binary_mask(prob_volum, output_format)
        
        # Get Detection Box adn add result
        det_pos, det_size, conf  = best_bb_bounding_boxes(bn_volum, prob_volum)  
        print("R:", det_pos, det_size, conf)
        detection_results.append( 
                [item['id']] + det_pos + det_size + [conf])     
        
        # Copy metadata from predited image
        
        
        bn_volum.CopyInformation(item['image'])
        
        # Save NRRD mask prediction
        mask_file = os.path.join(output_folder, output_file_template.format(item['id']))
        sitk.WriteImage(bn_volum, mask_file, useCompression=True )
        
        # Save file names for evaluation
        pred_files.append(mask_file)
        if 'mask_path' in item:
                gt_files.append(item['mask_path'])
                
                
# Save detection results
det_df = pd.DataFrame(detection_results, 
                      columns=['public_id', 'coordX', 
                      'coordY', 'coordZ', 'x_length', 
                      'y_length','z_length', 'probability'])
det_df.to_csv(pred_csv, index=False)

                
if len(gt_files) > 0:
        with open(os.path.join(output_folder,"cases.json"), "w") as fp:
                json.dump({"gt_files": gt_files,
                        "pred_files": pred_files,
                        "confidance_th": confidance_th},fp)
                

        

Dataset columns: Index(['case_id', 'label', 'data_path', 'mask_path'], dtype='object')


  0%|          | 0/20 [00:00<?, ?it/s]

(353, 682, 865)
0 10 20 20 25 86
binary 0 1
Prob 0.0 0.81861037
8
BB (490, 50, 0, 215, 215, 7)
After crop volum: 0.0 0.65465575
[597.5, 157.5, 3.5] [215, 215, 7] 0.65465575
BB (151, 67, 149, 106, 162, 20)
After crop volum: 0.0 0.71395844
[204.0, 148.0, 159.0] [106, 162, 20] 0.6606274
BB (20, 41, 166, 56, 179, 11)
After crop volum: 0.0 0.61576873
[48.0, 130.5, 171.5] [56, 179, 11] 0.61576873
BB (226, 139, 194, 69, 72, 11)
After crop volum: 0.0 0.6048045
[260.5, 175.0, 199.5] [69, 72, 11] 0.6048045
BB (236, 112, 217, 94, 86, 11)
After crop volum: 0.0 0.6476124
[283.0, 155.0, 222.5] [94, 86, 11] 0.64761245
BB (396, 76, 258, 116, 111, 20)
After crop volum: 0.0 0.81861037
[454.0, 131.5, 268.0] [116, 111, 20] 0.73504937
BB (236, 94, 268, 83, 104, 11)
After crop volum: 0.0 0.7656488
[277.5, 146.0, 273.5] [83, 104, 11] 0.7656488
BB (311, 112, 290, 57, 48, 11)
After crop volum: 0.0 0.66155297
[339.5, 136.0, 295.5] [57, 48, 11] 0.66155297
R: [273.5, 146.0, 277.5] [11, 104, 83] 0.7656488


  5%|▌         | 1/20 [00:25<08:07, 25.66s/it]

(353, 682, 865)
0 10 20 20 25 86
binary 0 1
Prob 0.0 0.8915669
5
BB (339, 121, 118, 88, 146, 22)
After crop volum: 0.0 0.88244075
[383.0, 194.0, 129.0] [88, 146, 22] 0.7912534
BB (716, 32, 126, 68, 135, 12)
After crop volum: 0.0 0.66344136
[750.0, 99.5, 132.0] [68, 135, 12] 0.6403458
BB (170, 67, 130, 156, 198, 27)
After crop volum: 0.0 0.8915669
[248.0, 166.0, 143.5] [156, 198, 27] 0.83090806
BB (283, 157, 169, 76, 108, 26)
After crop volum: 0.0 0.8483019
[321.0, 211.0, 182.0] [76, 108, 26] 0.75971174
BB (368, 183, 195, 35, 57, 11)
After crop volum: 0.0 0.60040885
[385.5, 211.5, 200.5] [35, 57, 11] 0.6004089
R: [143.5, 166.0, 248.0] [27, 198, 156] 0.83090806


...

1
BB (349, 57, 210, 160, 177, 44)
After crop volum: 0.0 0.86147785
[429.0, 145.5, 232.0] [160, 177, 44] 0.78008825
R: [232.0, 145.5, 429.0] [44, 177, 160] 0.78008825
(330, 608, 865)
0 9 18 18 25 86
binary 0 1
Prob 0.0 0.8494456


 65%|██████▌   | 13/20 [05:05<02:40, 23.00s/it]

1
BB (311, 110, 197, 88, 117, 19)
After crop volum: 0.0 0.8494456
[355.0, 168.5, 206.5] [88, 117, 19] 0.74278325
R: [206.5, 168.5, 355.0] [19, 117, 88] 0.74278325
(353, 682, 865)
0 10 20 20 25 86
binary 0 1
Prob 0.0 0.88564867
5
BB (188, 32, 82, 286, 300, 47)
After crop volum: 0.0 0.88564867
[331.0, 182.0, 105.5] [286, 300, 47] 0.8331987
BB (188, 139, 148, 67, 97, 11)
After crop volum: 0.0 0.62298095
[221.5, 187.5, 153.5] [67, 97, 11] 0.62298095
BB (358, 94, 151, 97, 126, 27)
After crop volum: 0.0 0.8847688
[406.5, 157.0, 164.5] [97, 126, 27] 0.80317867
BB (726, 121, 171, 58, 119, 11)
After crop volum: 0.0 0.6824741
[755.0, 180.5, 176.5] [58, 119, 11] 0.6824741
BB (443, 130, 214, 59, 101, 12)
After crop volum: 0.0 0.7963882
[472.5, 180.5, 220.0] [59, 101, 12] 0.7227142
R: [105.5, 182.0, 331.0] [47, 300, 286] 0.8331987


 70%|███████   | 14/20 [05:30<02:22, 23.73s/it]

(353, 682, 865)
0 10 20 20 25 86
binary 0 1
Prob 0.0 0.90316254


In [ ]:
det_df_sorted = det_df.sort_values(by = 'public_id')
pred_sorted_csv = os.path.join(output_folder, "prediction_sorted.csv")
det_df_sorted.to_csv(pred_sorted_csv, index=False)

In [ ]:
import json
with open(os.path.join(output_folder,"cases.json"), "w") as fp:
        json.dump({"gt_files": gt_files,
                "pred_files": pred_files,
                "confidance_th": confidance_th},fp)

#### Validation

In [ ]:
from TDSCABUS2023.Metrics import segmentation
from TDSCABUS2023.Metrics import detection


def Validate(pred_list, gt_list, cvs_pred_file = None, csv_gt_file = None):
    
    print("Segmentation:")
    print("------------------------------------------")
    
    scores = {'DiceCoefficient': [], 'HDCoefficient': [], 'score': []}
    for pred, gt in zip(pred_list, gt_list):
        try:
            result = segmentation.score_case(gt, pred)
        except Exception as e:
           result = {'DiceCoefficient': 0, 'HDCoefficient': 0, 'score': 0} #HD coefficient if fail?
        print("Case:", os.path.basename(pred), "  Results:",  result)
        
        for k, v in result.items():
            scores[k].append(v)
        
    for k, values in scores.items():
        values = np.array(values)
        print(f"\n{k}:")
        print(f"   - Min: {values.min():0.4f}")
        print(f"   - Max: {values.max():0.4f}")
        print(f"   - Mean: {values.mean():0.4f}")
    

In [ ]:
gt_files = [g.replace("DATA", "MASK") for g in gt_files]
print(pred_files)
print(gt_files)


['results_masks/abus23_25_test/raw_stack_train10_default/MASK_107.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_109.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_105.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_127.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_116.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_118.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_119.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_115.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_101.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_124.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_128.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_126.nii.gz', 'results_masks/abus23_25_test/raw_stack_train10_default/MASK_104.nii.gz', 'results_masks/abus23_25_test/raw_sta

Detection

In [ ]:
if "bbx_labels" in locals():
    # Addapt the cases for the etst
    temp_csv = "./temp_test.csv"

    val_ids = get_validation_ids(validation_file)
    gt_info = pd.read_csv(gt_csv)
    test_info = gt_info[gt_info.public_id.isin(val_ids)]
    print("Subset:\n------------------------------\n", test_info, "\n---------------------------------")
    test_info.to_csv(temp_csv, index=False)

    if os.path.exists(gt_csv):
        detection.eval_withprint(temp_csv, pred_csv)

Segmentation

In [ ]:
if "gt_files" not in locals() or "pred_files" not in locals():
    if not os.path.exists(os.path.join(output_folder,"cases.json")):
        print(f"Data not found in {output_folder}")
    print(f"Loading data from {output_folder}..")
    with open(os.path.join(output_folder,"cases.json")) as fp:
        json_data = json.load(fp)
        gt_files, pred_files = json_data["gt_files"], json_data["pred_files"]
        confidance_th = json_data["confidance_th"]

print(f"Validating.. (Conf: {confidance_th})")
Validate(pred_files, gt_files)


Validating.. (Conf: 0.6)
Segmentation:
------------------------------------------

DiceCoefficient:


ValueError: zero-size array to reduction operation minimum which has no identity